In [73]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [74]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../', '../', '../', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [75]:
x_train, y_train,x_test_original, y_test_original = ReadTS('FaceAll')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)

(560, 131, 1)


In [76]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    # 获取原始数据集的形状
    original_shape = TS.shape
    
    # 将数据集reshape成二维数组
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    # 初始化StandardScaler
    scaler = StandardScaler()
    
    # 对数据集进行标准化
    TS_normalized = scaler.fit_transform(TS_2d)
    
    # 将标准化后的数据集reshape回原始形状
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [100]:
model = make_CNN_model(x_train.shape[1:],num_classes = 14)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
14/14 [==============================] - 0s 14ms/step - loss: 2.4848 - sparse_categorical_accuracy: 0.1362 - val_loss: 2.7288 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/100
14/14 [==============================] - 0s 9ms/step - loss: 2.2449 - sparse_categorical_accuracy: 0.2054 - val_loss: 2.8346 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/100
14/14 [==============================] - 0s 9ms/step - loss: 2.1350 - sparse_categorical_accuracy: 0.2902 - val_loss: 3.0227 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/100
14/14 [==============================] - 0s 9ms/step - loss: 2.0595 - sparse_categorical_accuracy: 0.3817 - val_loss: 3.3064 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/100
14/14 [==============================] - 0s 10ms/step - loss: 1.9974 - sparse_categorical_accuracy: 0.4978 - val_loss: 3.6790 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/100
14/14 [==============================] - 0s 8ms/step - loss: 1.9583 - s

In [101]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

53/53 [==============================] - 0s 2ms/step - loss: 3.2148 - sparse_categorical_accuracy: 0.5485
Test accuracy 0.5485206842422485
Test loss 3.21478533744812


### Create perturbed test sets and calculate decrease in accuracy

In [102]:
print(y_test_original.shape)

(1690,)


In [103]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = [],bg = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1) 
    if perturbation_strategy == 'RBP1fast':
        return RBPIndividualNew1fast(bg, ts, index0, index1)  
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[], bg=[])


In [140]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 0
index_1 = 80
x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

bg = multisamplebackgroundIdentification(perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1fast',x,index_0,index_1, bg = bg) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

53/53 [==============================] - 0s 2ms/step


In [141]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

8.480121761858857
9.049227905729188
8.767810547552116
15.136256280993672
2.065721398595376


In [142]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [143]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [144]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.5485207100591716
 RBP Accuracy: 0.11124260355029586
 RBP1 Accuracy: 0.13550295857988165
 Zero Accuracy: 0.10828402366863905
 Noise Accuracy: 0.1
 Blur Accuracy: 0.4798816568047337



In [145]:
print(np.argmax(predictions_new1,axis=1))

[2 2 2 ... 4 2 4]
